# VIT

MS COCO 2014

## Imports

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, losses, callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import Sequence
from pycocotools.coco import COCO
import tensorflow as tf
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sys

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(gpus)
print(tf.__version__)

## Constantes et variables globals

In [ ]:
# Paths
ANNOTDIR = 'annotations_trainval2014'
DATADIR = 'train2014'
INSTANCEFILE = '{}/annotations/instances_{}.json'.format(ANNOTDIR, DATADIR)

# Hyper-paramètres
RATIO_TRAIN = 0.8
RATIO_VAL = 0.15
RATIO_TEST = 0.05
BATCH_SIZE = 32
EPOCHS = 200
PATIENCE = 3
EMBEDDING_DIM = 256
COCO_INSTANCES = COCO(INSTANCEFILE)
NUM_TOTAL_CLASSES = 90 # 80 classes + 10 classes omises

# Vérifications
assert RATIO_TRAIN + RATIO_VAL + RATIO_TEST == 1 # Vérification de la somme des ratios

num_classes = len(COCO_INSTANCES.getCatIds())
print(f'Nombre de classes dans le dataset COCO: {num_classes}')

## Chargement des données

In [ ]:
class DatasetGenerator(Sequence):
    def _getsplit(self, ensemble):
        if ensemble == 'train':
            start = 0
            stop = int(RATIO_TRAIN * len(self.imgIds))
        elif ensemble == 'val':
            start = int(RATIO_TRAIN * len(self.imgIds))
            stop = int((RATIO_TRAIN + RATIO_VAL) * len(self.imgIds))
        elif ensemble == 'test':
            start = int((RATIO_TRAIN + RATIO_VAL) * len(self.imgIds))
            stop = len(self.imgIds)
        return start, stop

    def __init__(self, ensemble, **kwargs):
        super().__init__(**kwargs)
        self.ensemble = ensemble
        
        # Créer une liste de tous les IDs d'images
        self.imgIds = COCO_INSTANCES.getImgIds()
        start, stop = self._getsplit(ensemble)
        self.ids = self.imgIds[start:stop]

    def __len__(self):
        return int(np.ceil(len(self.ids) / BATCH_SIZE))

    def __getitem__(self, index):
        batch_ids = self.ids[index * BATCH_SIZE : (index + 1) * BATCH_SIZE]
        batch_images = []
        batch_labels = []
        for id in batch_ids:
            # Charger l'image
            file_name = COCO_INSTANCES.imgs[id]['file_name']
            image = Image.open(f'{DATADIR}/{file_name}')
            image = image.resize((224, 224))
            image = image.convert('RGB')
            image = np.array(image)
            batch_images.append(image)
            # Charger les classes
            annIds = COCO_INSTANCES.getAnnIds(imgIds=id)
            anns = COCO_INSTANCES.loadAnns(annIds)
            labels = [0.0 for _ in range(NUM_TOTAL_CLASSES)]
            for ann in anns:
                labels[ann['category_id']] = 1.0
            batch_labels.append(labels)

        batch_labels = np.array(batch_labels)
        batch_images = np.array(batch_images) / 255.0

        return (batch_images, batch_labels)

    def on_epoch_end(self):
        self.ids = np.random.permutation(self.ids)

train_generator = DatasetGenerator('train')
val_generator = DatasetGenerator('val')
test_generator = DatasetGenerator('test')

print(f'Taille du dataset d\'entrainement: {len(train_generator)} batches, {len(train_generator.ids)} items')
print(f'Taille du dataset de validation: {len(val_generator)} batches, {len(val_generator.ids)} items')
print(f'Taille du dataset de test: {len(test_generator)} batches, {len(test_generator.ids)} items')

### Test unitaire du générateur de données

In [ ]:
generator = train_generator
# Récupérer un batch d'images et de légendes
r_index = np.random.randint(len(generator))
images, labels = generator.__getitem__(r_index-1)
# Extraire une image et ses classes
r_index = np.random.randint(len(images))
image = images[r_index]
label = labels[r_index]
label_ids= [str(i) for i in np.where(label == 1)[0]]
label_str = ', '.join([ COCO_INSTANCES.cats[int(i)]['name'] for i in label_ids])
# Afficher une image et ses classes
plt.imshow(image)
plt.title(f'Classes: {label_str}')
plt.axis('off')
plt.show()

## Modèle

### Création du modèle

In [ ]:

### CUSTOM LAYERS ###



### CUSTOM MODELS ###

def VIT_v1():
    '''
    inputs :
    - image : (224, 224, 3)
    outputs :
    - class : (80)
    '''
    # Création du modèle
    image_input = layers.Input(shape=(224, 224, 3))

    model = Model(inputs=image_input, outputs=output, name='VIT_v1')
model = VIT_v1()
model.summary()

#### Visualisation du modèle

In [ ]:
plot_model(model, to_file=f'{model.name}.png', show_shapes=True, show_layer_names=True)

### Entrainement du modèle

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         patience=PATIENCE, 
                                         restore_best_weights=True)

checkpoint_path = f'checkpoints/{model.name}'
checkpoint_path = checkpoint_path + '-{epoch:04d}.keras'
checkpoint_dir = os.path.dirname(checkpoint_path)
model_checkpoint = callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=EPOCHS,
                    callbacks=[early_stopping],
                    verbose=1)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

#### Sauvegarde du modèle

In [ ]:
model.save(f'Livrable3_{model.name}.keras')